In [ ]:
import os, io
import pytz
import urllib, json
import datetime as dt
import pandas as pd
import csv
from datetime import date, datetime

from sqlalchemy import create_engine, MetaData
from sqlalchemy.pool import NullPool
from sqlalchemy.sql import select, and_
from settings.db import DO_URL
from settings.apis import FORECAST_API_SETTINGS


engine = create_engine(DO_URL, poolclass=NullPool)
metadata = MetaData()
metadata.reflect(bind=engine)

In [ ]:
#Select here site to get forecasts for
from settings.sites import kherson as sites_list

query = f'''SELECT displayable_name, id, w_code FROM sites WHERE displayable_name in ({str(sites_list)[1:-1]})'''
print(query)
with engine.connect() as connection:
    sites = connection.execute(query).fetchall()

In [ ]:
#Set dates interval providing firt_date and last_date

first_date = dt.date(2023, 1, 1)
last_date = dt.date(2023, 1, 31)

In [ ]:
target_folder = 'data/forecasts/{}_{}/'.format(first_date.strftime('%Y-%m-%d'), last_date.strftime('%Y-%m-%d'))
print(target_folder)
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

In [ ]:
from uce_resources import get_applied_forecast

forecasts = dict()

with engine.connect() as connection:
    for site, site_id, w_code in sites:
        forecast = get_applied_forecast(site_id, first_date, last_date, connection, metadata.tables['forecasting_data'])
        
        forecasts.update({site: forecast})
        with pd.ExcelWriter(target_folder + f'{site}_forecast.xlsx', engine="openpyxl") as  writer:
            forecasts[site].to_excel(writer, 'results_daily')

## UTP Makets

In [ ]:
def get_fake_data_API(site_id, date, connection):
    date_str = date.strftime('%Y-%m-%d')
    data = get_applied_forecast(site_id, date, date, connection, metadata.tables['forecasting_data'])
    forecast = pd.DataFrame(index=range(1, len(data.index) + 1), data=data['forecast [kWh]'].to_list(), columns=[date_str])
    forecast['{}'.format(date_str)]
    forecast['Generation']=forecast['{}'.format(date_str)]
    num = forecast['Generation']._get_numeric_data()
    num[num < 0] = 0
    generation1 = forecast['Generation'].values.tolist()
    forecast['Consumption']=forecast['{}'.format(date_str)]*(-1)
    numb = forecast['Consumption']._get_numeric_data()
    numb[numb < 0] = 0
    consumption = forecast['Consumption']
    consumption = forecast['Consumption'].values.tolist()
    generation = generation1
    consurpation = consumption           
    return (generation, consurpation)


def create_row_list(sites, date, connection):
    row_list = list()
    row_list.append(['((//30817', '{}'.format(date.strftime('%m%d')), '38269062', '++'])
    for site, site_id, w_code in sites:
        row_data = get_fake_data_API(site_id, date, connection)
        consumption_list = ['({}_1)'.format(w_code)] + [sum(row_data[1])] + row_data[1] + ['']
        row_list.append(consumption_list)        
        generation_list = ['({}_2)'.format(w_code)] + [sum(row_data[0])] + row_data[0] + ['']
        row_list.append(generation_list)
    row_list.append(['==))'])
    return row_list

enter_date = dt.date(2023, 1, 31)

target_folder = 'data/utp_mock-up/'
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

with engine.connect() as connection:
    row_list = create_row_list(sites=sites, date=enter_date, connection=connection)

with open(target_folder + 'forecast_{}_{}.txt'.format('kherson', enter_date.strftime('%m%d')), 'w', newline='') as file:
    writer = csv.writer(file, quoting=csv.QUOTE_NONE, delimiter=':')
    writer.writerows(row_list)
print('UTP data created!')